# **Problem Set 4**

## **To keep in mind when completing the assignment: You should enter formulas when you are doing calculations, no rounded values.**  
_Doing the calculations manually with a calculator and then enter the value might return your answer as incorrect if your rounding is not the same as the one from the autograding._

## **Tasks**
- For part 1, you should answer the multiple choice questions about the differences between the substitution approach and the partitioning approach, and pros and cons of both approaches.
- For part 2, apply substitution allocation for electricity and heat production respectively, and partitioning based on energy and cost, using equations.
- For part 3, construct and show the Aff, Abf and Sf matrices for electricity and heat production respectively, and partitioning based on energy and cost. Calculate the different emissions for all four cases.

## **Passing requirement**
- **At least 80 points**

**_Please run the cell below before you start the assignment_**

In [1]:
# Import required Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Folder for the files
files_folder = "files_PS4_V1_0/"

### In case formulas are entered in the csv files & correcting the datatypes
def convert_formulas(matrix):
    for row in matrix.index:
        for column in matrix.columns:
            if type(matrix.loc[row, column]) == str:
                if " " in matrix.loc[row, column]:
                    matrix.loc[row, column] = matrix.loc[row, column].lstrip(' ')
                if "=" in matrix.loc[row, column]:
                    matrix.loc[row, column] = matrix.loc[row, column].lstrip('=')
                matrix.loc[row, column] = pd.eval(matrix.loc[row, column])
    ### Get the data types right in the DataFrames
    s = matrix.select_dtypes(include='object').columns
    matrix[s] = matrix[s].astype("float64")
    return matrix

## **Combined heat and power (CHP) Problem**

Consider a combined heat and power (CHP) plant which uses natural gas as its input. This is a process that produces two joint products that cannot be disaggregated effectively. Thus, allocation is necessary in order to understand the impact due to each unit of its products.

Assume this CHP has an overall efficiency of 87 per cent. Hence, for every kilowatt-hour (kWh) of natural gas input, it produces 0.31 kWh of electricity and 0.56 kWh of heat.

The direct CO$_2$ emissions per kWh of electricity and heat from a natural gas fired CHP are 230 g CO$_2$ [1] (i.e. per kWh of useful energy output). Assume that there are no other emissions. Your task is to find the GWP allocated to the production of one unit of electricity and one unit of heat from the CHP plant, based on the information given below.

References: 
[1] Nilsson, M. and Stromberg, P, 2000, Evaluation of different methods for allocation in co-generation, Final Report

The figure below presents the CHP system - it is without numbers but gives an idea of how the system works:

<img src="files_PS4_V1_0/figure_CHP_nonumbers.png" width="700">


## **Part 1. Allocation theory** _(20 points)_

In this section, you have to answer multiple choice questions to show that you understand (1) what is meant with allocation, (2) the difference between the substitution approach and the partitioning approach and (3) give pros and cons of both approaches.

As a point of reference you may want to look into what the ISO 14044:2006 guidelines (section 4.3.4 Allocation) say about allocation procedures in LCA on pg. 24 section 4.3.4.2. You can access this document at https://i.ntnu.no/wiki/-/wiki/English/Standards in the database ‘Standard.no’.

#### **1a)** _(5 points)_ Which one(s) of these statement(s) is/are correct?

1. Allocation means dividing the emissions from a process producing more than one product on the different products.
2. Allocation means giving the spatial location of different products produced by the process production.
3. Allocation is a method for handling by-products.
 
Define your answer as the variable `answer_1a` (e.g. `answer_1a = {1}` if you think the correct statement is statement {1} and `answer_1a = {1, 2}` if you think the correct statements are statements {1, 2}).

In [2]:
answer_1a = {1, 3}

**_Run the cell under to check the format of your answer(s)_**

In [3]:
assert all (type(i) in [set] for i in [answer_1a]), 'The answer(s) must be a set of values'

#### **1b)** _(5 points)_ Substitution or partitioning?

If we assume:
1. Substitution
2. Partitioning

Which type of allocation "divides the emissions by assuming that one of the products is produced using an alternative technology."?

Define your answer as the variable `answer_1b` (e.g. `answer_1b = {1}` if you think the correct answer is "1. Substitution" and `answer_1b = {2}` if you think the correct answer is "2. Partitioning").

In [4]:
answer_1b = {1}

**_Run the cell under to check the format of your answer(s)_**

In [5]:
assert all (type(i) in [set] for i in [answer_1b]), 'The answer(s) must be a set of values'

#### **1c)** _(5 points)_ Substitution or partitioning?

If we assume:
1. Substitution
2. Partitioning

Which type of allocation "divides the emissions based on an attribute of the products, e.g. energy content or price."?

Define your answer as the variable `answer_1c` (e.g. `answer_1c = {1}` if you think the correct answer is "1. Substitution" and `answer_1c = {2}` if you think the correct answer is "2. Partitioning").

In [6]:
answer_1c = {2}

**_Run the cell under to check the format of your answer(s)_**

In [7]:
assert all (type(i) in [set] for i in [answer_1c]), 'The answer(s) must be a set of values'

#### **1d)** _(5 points)_ Which one(s) of these statement(s) is/are correct?

1. Results may differ very much based on which allocation method is chosen.
2. When applying substitution allocation, it is crucial to select relevant products/processes to substitute original products/processes from the system under investigation.
3. Substitution might be misused for greenwashing.
4. Substitution may lead to negative emissions for the remaining process.
5. For partitioning, it is important to know the system analysed to select the correct / most relevant partitioning coefficient, i.e. one that best represents the driving force of the multiple production.
 
Define your answer as the variable `answer_1d` (e.g. `answer_1d = {1}` if you think the correct statement is statement {1} and `answer_1d = {1, 2}` if you think the correct statements are statements {1, 2}).

In [8]:
answer_1d = {1, 2, 3, 4, 5}

**_Run the cell under to check the format of your answer(s)_**

In [9]:
assert all (type(i) in [set] for i in [answer_1d]), 'The answer(s) must be a set of values'

## **Part 2. Principles and scalar form - without background processes** _(40 points)_

Use the information given under each sub-task (2a.-2d.) to determine the global warming potential (GWP) of producing 1 kWh of electricity and 1 kWh of heat using formulas from the reader.

_**You have to write the calculations you are performing directly in the cells.** Doing the calculations manually with a calculator and then enter the value might return your answer as incorrect if your rounding is not the same as the one from the autograding._

#### **2a)** _(10 points)_ Substitution of heat - Impact of 1 kWh of electricity from CHP

Follow the description under and calculate the impact (in kg CO$_2$-eq) of 1 kWh of electricity from the CHP. Define your answer as `answer_2a`.

a. Apply substitution allocation to find the impacts due to 1 kWh of electricity production from the CHP. An alternative heat source – a light fuel oil furnace with overall efficiency of heat production 85 per cent – is used to calculate the impacts associated with heat production. It has direct CO$_2$ emissions of 299 gCO$_2$-eq/kWh$_{heat}$ [2] (that is per kWh of useful heat).

References:
[2] From Eco-Invent Database

In [10]:
u_che = 0.23
u_h = 0.299
x_h = 0.56/0.31
x_che = 1 + x_h

answer_2a = u_che*x_che - u_h*x_h
print(answer_2a)

0.10535483870967743


**_Run the cell under to check the format of your answer(s)_**

In [11]:
assert all (type(i) in [float] for i in [answer_2a]), 'The answer(s) must be a float'

#### **2b)** _(10 points)_ Substitution of electricity - Impact of 1 kWh of heat from CHP

Follow the description under and calculate the impact (in kg CO$_2$-eq) of 1 kWh of heat from the CHP. Define your answer as `answer_2b`.

b. Apply substitution allocation to find the impacts due to 1 kWh of heating production from the CHP. Wind power is used as an alternative electricity source. A 1.5MW (on-shore) wind turbine producing the same amount of electricity is used to evaluate the CO$_2$ emissions with respect to electricity production. The specific life cycle CO$_2$ emissions from wind power are 10.5 gCO2$_2$-eq/kWh$_{elec}$ [3]. 

References:
[3] Pehnt, M., 2006, Dynamic life cycle assessment (LCA) of renewable energy technologies, Renewable Energy, 31, pp. 55-71.

In [12]:
u_che = 0.23
u_e = 0.0105
x_e = 0.31/0.56
x_che = 1 + x_e

answer_2b = u_che*x_che - u_e*x_e
print(answer_2b)

0.35150892857142857


**_Run the cell under to check the format of your answer(s)_**

In [13]:
assert all (type(i) in [float] for i in [answer_2b]), 'The answer(s) must be a float'

#### **2c)** _(10 points)_ Partitioning based on energy - Impact of 1 kWh of electricity and 1 kWh of heat from CHP

Follow the description under and calculate the impacts (in kg CO$_2$-eq) of 1 kWh of electricity and 1 kWh of heat from the CHP. Define your answers as `answer_2c_elec` and `answer_2c_heat`.

c. Apply partitioning based on energy to find the impact due to 1 kWh of electricity and 1 kWh of heating from the CHP.

In [14]:
a = 0.31/0.87

#answer_2c_elec = 0.230 * 0.31/0.87 * 1/0.31
answer_2c_elec = a * (0.87/0.31) * 0.230
print(answer_2c_elec)

#answer_2c_heat = 0.230 * 0.56/0.87 * 1/0.56
answer_2c_heat = (1-a) * 0.230 * (0.87/0.56)
print(answer_2c_heat)

0.23
0.22999999999999995


**_Run the cell under to check the format of your answer(s)_**

In [15]:
assert all (type(i) in [float] for i in [answer_2c_elec, answer_2c_heat]), 'The answer(s) must be a float'

#### **2d)** _(10 points)_ Partitioning based on cost - Impact of 1 kWh of electricity and 1 kWh of heat from CHP

Follow the description under and calculate the impacts (in g CO$_2$-eq) of 1 kWh of electricity and 1 kWh of heat from the CHP. Define your answers as `answer_2d_elec` and `answer_2d_heat`.

d. Apply partitioning based on costs to find the impact due to 1 kWh of electricity and 1 kWh of heating from the CHP – cost of electricity produced = 1 NOK/KWh$_{elec}$ and the cost of heat produced = 0.7 NOK/KWh$_{heat}$.

In [16]:
c = 1*0.31 + 0.7*0.56

answer_2d_elec = 0.230 * (0.31/c) * (0.87/0.31)
print(answer_2d_elec)

answer_2d_heat = 0.230 * ((0.7*0.56)/1.7) * (0.87/0.56)
print(answer_2d_heat)

0.2850427350427351
0.08239411764705881


**_Run the cell under to check the format of your answer(s)_**

In [17]:
assert all (type(i) in [float] for i in [answer_2d_elec, answer_2d_heat]), 'The answer(s) must be a float'

## **Part 3. Matrix form - with background processes** _(40 points)_

Use the information from the descriptions in the previous part (a.-d.) with the background matrices in **files_PS4_V1_0**. Assume the only background to foreground input is the energy input needed to produce the heat and power (i.e. natural gas for the CHP and fuel oil for the oil furnace). Assume zero procurement losses, and no fuel transport, plant capital or plant decommissioning requirements. Be aware of units: 3.6 MJ/kWh!

We need A and S matrices for all the four problems. When we have established these, we calculate the total emissions d, like we have done in previous problem sets.

Some matrices/vectors are already given to you:
- `Afb_Abb` which contains Afb and Abb: it is the same one which is used in all the cases
- `Sb`: it is the same one which is used in all the cases
- `C`: it is the same one which is used in all the cases

You have to fill in 8 matrices and 2 vectors which are the following:
- substitution of heat: `Aff_Abf_subheat` which contains Aff and Abf & `Sf_subheat` which contains Sf
- substitution of electricity: `Aff_Abf_subel` which contains Aff and Abf & `Sf_subel` which contains Sf
- partitioning based on energy: `Aff_Abf_parten` which contains Aff and Abf & `Sf_parten` which contains Sf
- partitioning based on cost: `Aff_Abf_partcost` which contains Aff and Abf & `Sf_partcost` which contains Sf
- `yheat` & `yelec`: the vectors containing the final demand of heat and electricity

For filling in these matrices, you have two options: 
- Option 1: filling in the matrices in the csv files located in the **files_PS4_V1_0** folder and save the csv files by using "ctrl+s". **You should enter the formulas in the csv files.**
- Option 2: filling in the matrices in the cells in the sections 3a), 3b), 3c), 3d). If you go for that option, we recommend you to make a quick hand sketch of the matrices to visualize how they should look like. **You have to write the frmulas you are using.**

_**Run the cell under to import the matrices and vectors**_

In [29]:
### Given
Afb_Abb = convert_formulas(pd.read_csv(files_folder + 'Afb_Abb.csv', index_col = [0]))
Sb = convert_formulas(pd.read_csv(files_folder + 'Sb.csv', index_col = [0]))
C = convert_formulas(pd.read_csv(files_folder + 'C.csv', index_col = [0]))

### To fill in - either in csv files or in 3a), 3b), 3c), 3d)
Aff_Abf_subheat = convert_formulas(pd.read_csv(files_folder + 'Aff_Abf_subheat.csv', index_col = [0]))
Sf_subheat = convert_formulas(pd.read_csv(files_folder + 'Sf_subheat.csv', index_col = [0]))
Aff_Abf_subel = convert_formulas(pd.read_csv(files_folder + 'Aff_Abf_subel.csv', index_col = [0]))
Sf_subel = convert_formulas(pd.read_csv(files_folder + 'Sf_subel.csv', index_col = [0]))
Aff_Abf_parten = convert_formulas(pd.read_csv(files_folder + 'Aff_Abf_parten.csv', index_col = [0]))
Sf_parten = convert_formulas(pd.read_csv(files_folder + 'Sf_parten.csv', index_col = [0]))
Aff_Abf_partcost = convert_formulas(pd.read_csv(files_folder + 'Aff_Abf_partcost.csv', index_col = [0]))
Sf_partcost = convert_formulas(pd.read_csv(files_folder + 'Sf_parten.csv', index_col = [0]))
yheat = convert_formulas(pd.read_csv(files_folder + 'yheat.csv', index_col = [0]))
yelec = convert_formulas(pd.read_csv(files_folder + 'yel.csv', index_col = [0]))

Fill in the two vectors `yheat` & `yelec`. We ask to fill in final demand of heat and electricity from the foreground processes (ie) 'Heat (Kwh)' and 'El (kWh)'.

In [30]:
yheat = yheat.fillna(0)
yelec = yelec.fillna(0)

display(yheat)
display(yelec)

,Final demand
CHP (kWh),0.0
Heat (kWh),1.0
El (kWh),0.0
Electricity (N. Gas kWh),0.0
Steel (kg),0.0
Aluminium (kg),0.0
Platinum (kg),0.0
Plastic (kg),0.0
Diesel (MJ),0.0
N. Gas (MJ),0.0


,Final demand
CHP (kWh),0.0
Heat (kWh),0.0
El (kWh),1.0
Electricity (N. Gas kWh),0.0
Steel (kg),0.0
Aluminium (kg),0.0
Platinum (kg),0.0
Plastic (kg),0.0
Diesel (MJ),0.0
N. Gas (MJ),0.0


**_Run the cell under to check the format of your answer(s)_**

In [20]:
assert (yheat.index == ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)', 'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)', 'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)']).all(), 'The index do not correspond'
assert (yheat.columns == ['Final demand']).all(), 'The columns do not correspond'
assert (yelec.index == ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)', 'Platinum (kg)', 'Plastic (kg)','Diesel (MJ)', 'N. Gas (MJ)', 'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)']).all(), 'The index do not correspond'
assert (yelec.columns == ['Final demand']).all(), 'The columns do not correspond'

#### **3a)** _(10 points)_ Substitution of heat - Impact of 1 kWh of electricity from CHP

Follow the description under and calculate the vector d of impact of 1 kWh of electricity from the CHP using the matrix form. Define your answer as `d_3a` and rename its column as "Impact". Remember that you have to do all the calculations leading to the vector of total impact d!

a. Apply substitution allocation to find the impacts due to 1 kWh of electricity production from the CHP. An alternative heat source – a light fuel oil furnace with overall efficiency of heat production 85 per cent – is used to calculate the impacts associated with heat production. It has direct CO$_2$ emissions of 299 gCO$_2$-eq/kWh$_{heat}$ [2] (that is per kWh of useful heat). 

References:
[2] From Eco-Invent Database

Assume diesel and light fuel oil are interchangeable when connecting energy requirements from the Abb matrix.

In [21]:
Aff_Abf_subheat = Aff_Abf_subheat.fillna(0)
#display(Aff_Abf_subheat)
A_subheat = pd.concat([Aff_Abf_subheat, Afb_Abb], axis = 1)
#display(A_subheat)

I = np.identity(13)
Larr = np.linalg.inv(I-A_subheat)
L = pd.DataFrame(Larr, index = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'], columns = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'])
#display(L)

x = L @ yelec
x = x.rename(columns = {'Final demand' : 'Output'})
#display(x)

Sf_subheat = Sf_subheat.fillna(0)
S_subheat = pd.concat([Sf_subheat, Sb], axis = 1)
#display(S_subheat)

d_3a = C @ (S_subheat @ x)
d_3a = d_3a.rename(columns = {'Output' : 'Impact'})
display(d_3a)

,Impact
GWP,-0.917397


**_Run the cell under to check the format of your answer(s)_**

In [22]:
assert (d_3a.index == ['GWP']).all(), 'The index do not correspond'
assert (d_3a.columns == ['Impact']).all(), 'The columns do not correspond'

#### **3b)** _(10 points)_ Substitution of electricity - Impact of 1 kWh of heat from CHP

Follow the description under and calculate the vector d of impact of 1 kWh of heat from the CHP using the matrix form. Define your answer as `d_3b` and rename its column as "Impact". Remember that you have to do all the calculations leading to the vector of total impact d!

b. Apply substitution allocation to find the impacts due to 1 kWh of heating production from the CHP. Wind power is used as an alternative electricity source. A 1.5MW (on-shore) wind turbine producing the same amount of electricity is used to evaluate the CO$_2$ emissions with respect to electricity production. The specific life cycle CO$_2$ emissions from wind power are 10.5 gCO2$_2$-eq/kWh$_{elec}$ [3]. 

References:
[3] Pehnt, M., 2006, Dynamic life cycle assessment (LCA) of renewable energy technologies, Renewable Energy, 31, pp. 55-71.

Since no disaggregated details are given with regards to embodied emissions per kWh$_{elec}$, insert the 10.5g/kWh$_{elec}$ in the Sf matrix in the column corresponding to wind power production.


In [23]:
Aff_Abf_subel = Aff_Abf_subel.fillna(0)
#display(Aff_Abf_subheat)
A_subel = pd.concat([Aff_Abf_subel, Afb_Abb], axis = 1)
#display(A_subheat)

I = np.identity(13)
Larr = np.linalg.inv(I-A_subel)
L = pd.DataFrame(Larr, index = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'], columns = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'])
#display(L)

x = L @ yheat
x = x.rename(columns = {'Final demand' : 'Output'})
#display(x)

Sf_subel = Sf_subel.fillna(0)
S_subel = pd.concat([Sf_subel, Sb], axis = 1)
#display(S_subheat)

d_3b = C @ (S_subel @ x)
d_3b = d_3b.rename(columns = {'Output' : 'Impact'})
display(d_3b)

,Impact
GWP,0.373732


**_Run the cell under to check the format of your answer(s)_**

In [24]:
assert (d_3b.index == ['GWP']).all(), 'The index do not correspond'
assert (d_3b.columns == ['Impact']).all(), 'The columns do not correspond'

#### **3c)** _(10 points)_ Partitioning based on energy - Impact of 1 kWh of electricity and 1 kWh of heat from CHP

Follow the description under and calculate the vector d of impact of 1 kWh of heat and the vector d of impact of 1 kWh of electricity from the CHP using the matrix form. Define your answer as `d_3c_elec` and `d_3c_heat` and rename their column as "Impact". Remember that you have to do all the calculations leading to the vector of total impact d!

c. Apply partitioning based on energy to find the impact due to 1 kWh of electricity and 1 kWh of heating from the CHP.

In [25]:
Aff_Abf_parten = Aff_Abf_parten.fillna(0)
#display(Aff_Abf_subheat)
A_parten = pd.concat([Aff_Abf_parten, Afb_Abb], axis = 1)
#display(A_subheat)

I = np.identity(13)
Larr = np.linalg.inv(I-A_parten)
L = pd.DataFrame(Larr, index = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'], columns = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'])
#display(L)

x_el = L @ yelec
x_el = x_el.rename(columns = {'Final demand' : 'Output'})

x_heat = L @ yheat
x_heat = x_heat.rename(columns = {'Final demand' : 'Output'})
#display(x)

Sf_parten = Sf_parten.fillna(0)
S_parten = pd.concat([Sf_parten, Sb], axis = 1)
#display(S_parten)

d_3c_elec = C @ (S_parten @ x_el)
d_3c_elec = d_3c_elec.rename(columns = {'Output' : 'Impact'})
display(d_3c_elec)

d_3c_heat = C @ (S_parten @ x_heat)
d_3c_heat = d_3c_heat.rename(columns = {'Output' : 'Impact'})
display(d_3c_heat)

,Impact
GWP,0.244305


,Impact
GWP,0.244305


**_Run the cell under to check the format of your answer(s)_**

In [26]:
assert (d_3c_heat.index == ['GWP']).all(), 'The index do not correspond'
assert (d_3c_heat.columns == ['Impact']).all(), 'The columns do not correspond'
assert (d_3c_elec.index == ['GWP']).all(), 'The index do not correspond'
assert (d_3c_elec.columns == ['Impact']).all(), 'The columns do not correspond'

#### **3d)** _(10 points)_ Partitioning based on cost - Impact of 1 kWh of electricity and 1 kWh of heat from CHP

Follow the description under and calculate the vector d of impact of 1 kWh of heat and the vector d of impact of 1 kWh of electricity from the CHP using the matrix form. Define your answer as `d_3d_elec` and `d_3d_heat` and rename their column as "Impact". Remember that you have to do all the calculations leading to the vector of total impact d!

d. Apply partitioning based on costs to find the impact due to 1 kWh of electricity and 1 kWh of heating from the CHP – cost of electricity produced = 1 NOK/KWh$_{elec}$ and the cost of heat produced = 0.7 NOK/KWh$_{heat}$.

In [31]:
Aff_Abf_partcost = Aff_Abf_partcost.fillna(0)
#display(Aff_Abf_subheat)
A_partcost = pd.concat([Aff_Abf_partcost, Afb_Abb], axis = 1)
#display(A_subheat)

I = np.identity(13)
Larr = np.linalg.inv(I-A_partcost)
L = pd.DataFrame(Larr, index = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'], columns = ['CHP (kWh)', 'Heat (kWh)', 'El (kWh)', 'Electricity (N. Gas kWh)', 'Steel (kg)', 'Aluminium (kg)',
       'Platinum (kg)', 'Plastic (kg)', 'Diesel (MJ)', 'N. Gas (MJ)',
       'Transport (Truck, tkm)', 'Concrete (kg)', 'Glass (kg)'])
#display(L)

x_el = L @ yelec
x_el = x_el.rename(columns = {'Final demand' : 'Output'})

x_heat = L @ yheat
x_heat = x_heat.rename(columns = {'Final demand' : 'Output'})
#display(x)

Sf_parten = Sf_parten.fillna(0)
S_parten = pd.concat([Sf_parten, Sb], axis = 1)
#display(S_parten)

d_3d_elec = C @ (S_parten @ x_el)
d_3d_elec = d_3d_elec.rename(columns = {'Output' : 'Impact'})
display(d_3d_elec)

d_3d_heat = C @ (S_parten @ x_heat)
d_3d_heat = d_3d_heat.rename(columns = {'Output' : 'Impact'})
display(d_3d_heat)

,Impact
GWP,0.302771


,Impact
GWP,0.211939


**_Run the cell under to check the format of your answer(s)_**

In [28]:
assert (d_3d_heat.index == ['GWP']).all(), 'The index do not correspond'
assert (d_3d_heat.columns == ['Impact']).all(), 'The columns do not correspond'
assert (d_3d_elec.index == ['GWP']).all(), 'The index do not correspond'
assert (d_3d_elec.columns == ['Impact']).all(), 'The columns do not correspond'